## Here the algorithm of **regret matching** for Rock-Paper-Scissors to find the best response strategy to the strategy of the opponent is released

In [80]:
import random

random.seed(0)

Define constants:

In [81]:
ROCK, PAPER, SCISSORS = 0, 1, 2
NUM_ACTIONS = 3

regretSum = [0.0] * NUM_ACTIONS
strategy = [0.0] * NUM_ACTIONS
strategySum = [0.0] * NUM_ACTIONS

# Choose a strategy for the opponent
oppStrategy = [0.4, 0.3, 0.3]

Function to get current mixed strategy through regret-matching:

In [82]:
def get_strategy():
    normalizing_sum = 0
    for action in range(NUM_ACTIONS):
        strategy[action] = max(0, regretSum[action])
        normalizing_sum += strategy[action]

    for action in range(NUM_ACTIONS):
        if normalizing_sum > 0:
            strategy[action] /= normalizing_sum
        else:
            # If we don't regret about anything
            strategy[action] = 1.0 / NUM_ACTIONS
        strategySum[action] += strategy[action]

    return strategy

Function to get random action according to mixed-strategy distribution:

In [83]:
def get_action(strategy):
    # Next time for this function numpy will be used
    r = random.random()
    action = 0
    cumulativeProbability = 0
    while action < NUM_ACTIONS - 1:
        cumulativeProbability += strategy[action]
        if r < cumulativeProbability:
            break
        action += 1
    return action


Function to get average mixed strategy across all training iterations

In [84]:
def get_average_strategy():
    avg_strategy = [0.0] * NUM_ACTIONS
    normalizing_sum = sum(strategySum)
    for action in range(NUM_ACTIONS):
        if normalizing_sum > 0:
            avg_strategy[action] = strategySum[action] / normalizing_sum
        else:
            avg_strategy[action] = 1.0 / NUM_ACTIONS
    return avg_strategy


With these building blocks in place, we can now construct our training algorithm:

In [85]:
def train(iterations):
    for _ in range(iterations):

        # Get regret-matched mixed strategy actions
        strategy = get_strategy()
        myAction = get_action(strategy)
        otherAction = get_action(oppStrategy)

        # Compute action utilities

        # Method from guide book is a bit unreadable
        # actionUtility = [0.0] * NUM_ACTIONS
        # actionUtility[otherAction] = 0
        # actionUtility[(otherAction + 1) % NUM_ACTIONS] = 1
        # actionUtility[(otherAction - 1) % NUM_ACTIONS] = -1

        # We can rewrite it using the matrix of utility (represented in notes)
        # myAction - vertical, otherAction - horizontal, RPS accordingly to the order
        actionUtility = [[0, -1, 1],
                         [1, 0, -1],
                         [-1, 1, 0]]

        # Accumulate action regrets
        for action in range(NUM_ACTIONS):
            # In the guide book the following line is used, but with it strategy converges to [0, 1, 0]
            # Which means that we always play paper... It doesn't seem to be correct
            regretSum[action] += (actionUtility[action][otherAction] - actionUtility[myAction][otherAction])

            # But if suppose that we don't add negative regrets to the sum,
            # The strategy goes to [0.33, 0.36, 0.31]
            # This seems to be more correct
            # regretSum[action] += max(0 , actionUtility[action][otherAction] - actionUtility[myAction][otherAction])

        # print(regretSum)

Now we are ready to run the computation of the response strategy:

In [86]:
train(iterations=10000)
print(get_average_strategy())

[0.0022815968170315994, 0.9933221881942534, 0.004396214988714989]


In [87]:
# Let's compare these two methods: whether we learn on both wins and losses or only on losses
actionUtility = [[0, -1, 1],
                 [1, 0, -1],
                 [-1, 1, 0]]

strategyWL = [0.0022815968170315994, 0.9933221881942534, 0.004396214988714989]
strategyL = [0.32656612190646006, 0.36233503087561797, 0.31109884721792197]

expUtilityWL = 0
expUtilityL = 0

for i in range(NUM_ACTIONS):
    for j in range(NUM_ACTIONS):
        expUtilityWL += strategyWL[i] * oppStrategy[j] * actionUtility[i][j]
        expUtilityL += strategyL[i] * oppStrategy[j] * actionUtility[i][j]
        
print(f' Expected utility if we consider wins and losses: {expUtilityWL:0.3f}')
print(f' Expected utility if we consider only losses: {expUtilityL:0.3f}')


 Expected utility if we consider wins and losses: 0.099
 Expected utility if we consider only losses: 0.005


So we can conclude that it is better to consider wins and losses for regret minimization, because the expected utility will be higher